In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import polars as pl

In [3]:
data_path = "data.05_24_11_31"
task = "text_summarization"

# likelihood agnostic

In [4]:
df = pl.scan_ndjson(f"{data_path}/{task}_score.txt")
ldf = df.filter(pl.col("watermark_processor").str.contains("Gumbel"))\
        .select(["best_score", "la_score"])\
        .explode("best_score", "la_score")\
        .with_columns(pl.lit("DeltaGumbel").alias("wp"))\
        .groupby(["wp"])\
        .agg(
        [
                pl.count().alias("token_count"),
                pl.col("best_score").mean().alias("score_per_token"),
                pl.col("best_score").std().alias("score_per_token_std"),
                pl.col("la_score").mean().alias("la_score_per_token"),
                pl.col("la_score").std().alias("la_score_per_token_std"),
        ]
        )


In [5]:
ldf.collect()

wp,token_count,score_per_token,score_per_token_std,la_score_per_token,la_score_per_token_std
str,u32,f64,f64,f64,f64
"""DeltaGumbel""",883309,0.87653,1.444565,0.078438,0.776177


In [6]:
task = "machine_translation"

In [7]:
df = pl.scan_ndjson(f"{data_path}/{task}_score.txt",infer_schema_length=10000)
ldf = df.filter(pl.col("watermark_processor").str.contains("Gumbel"))\
        .select(["best_score", "la_score"])\
        .explode("best_score", "la_score")\
        .with_columns(pl.lit("DeltaGumbel").alias("wp"))\
        .groupby(["wp"])\
        .agg(
        [
                pl.count().alias("token_count"),
                pl.col("best_score").mean().alias("score_per_token"),
                pl.col("best_score").std().alias("score_per_token_std"),
                pl.col("la_score").mean().alias("la_score_per_token"),
                pl.col("la_score").std().alias("la_score_per_token_std"),
        ]
        )


In [8]:
ldf.collect()

wp,token_count,score_per_token,score_per_token_std,la_score_per_token,la_score_per_token_std
str,u32,f64,f64,f64,f64
"""DeltaGumbel""",70102,0.429649,1.171952,-0.104224,0.890775
